In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import requests
import pandas as pd
from io import StringIO
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

#load train data
import pandas as pd

import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

raw_datasets  = load_dataset('SemEvalWorkshop/humicroedit', 'subtask-1')



/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification

model_name = "answerdotai/ModernBERT-base"

#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'

In [3]:
from datasets import DatasetDict

mask_token = tokenizer.mask_token

def generate_prompt(data_point):
    """
    Generates a prompt for evaluating the humor intensity of an edited headline.
    Args:
        data_point (dict): A dictionary containing 'original', 'edit', and 'meanGrade'.
    Returns:
        str: The formatted prompt as a string.
    """
    return f"""# Original Headline: {data_point['original']}. # Edited Headline: {data_point['edit']} # Output: The mean funniness score is{mask_token}"""  # noqa: E501


# Assuming `dataset` is your DatasetDict
def add_label_column(example):

    example['labels'] = float(example['meanGrade'])
  
    example['input'] = generate_prompt(example)

    
    return example

# Map the function over train and validation datasets

train_data = raw_datasets['train'].map(add_label_column)
val_data = raw_datasets['validation'].map(add_label_column)

# Remove unnecessary columns

# Inspect the updated datasets
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)

Train Dataset: Dataset({
    features: ['id', 'original', 'edit', 'grades', 'meanGrade', 'labels', 'input'],
    num_rows: 9652
})
Validation Dataset: Dataset({
    features: ['id', 'original', 'edit', 'grades', 'meanGrade', 'labels', 'input'],
    num_rows: 2419
})


In [4]:
train_data

Dataset({
    features: ['id', 'original', 'edit', 'grades', 'meanGrade', 'labels', 'input'],
    num_rows: 9652
})

In [5]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer.padding_side = 'left'


# col_to_delete = ['idx']
col_to_delete =  ['id', 'original', 'edit', 'grades', 'meanGrade', 'input']

mask_token = tokenizer.mask_token
def preprocessing_function(examples):
   
    return tokenizer(examples['input'], truncation=True, max_length=512)

tokenized_train_data = train_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_val_data = val_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
tokenized_train_data.set_format("torch")
tokenized_val_data.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/2419 [00:00<?, ? examples/s]

Map: 100%|██████████| 2419/2419 [00:00<00:00, 24888.75 examples/s]


In [6]:
tokenizer.decode(tokenized_train_data['input_ids'][10])


'[CLS]# Original Headline: Dozens dead in possible gas <attack/> in Syria ; regime denies allegation. # Edited Headline: bloating # Output: The mean funniness score is[MASK][SEP]'

In [7]:
val_data

Dataset({
    features: ['id', 'original', 'edit', 'grades', 'meanGrade', 'labels', 'input'],
    num_rows: 2419
})

In [8]:
all_lengths = [len(ids) for ids in tokenized_train_data['input_ids']]
mx = max(all_lengths)
mx


79

In [9]:
count = sum(len(ids) > 512 for ids in tokenized_train_data['input_ids'])
print(count)


0


In [10]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification
from transformers.activations import ACT2FN
import random
from modeling import MLMSequenceClassification

config = AutoConfig.from_pretrained(model_name)

model = MLMSequenceClassification.from_pretrained(model_name, config=config, num_labels=1, mask_token_id=tokenizer.mask_token_id)



You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [11]:
model

MLMSequenceClassification(
  (transformer): ModernBertForMaskedLM(
    (model): ModernBertModel(
      (embeddings): ModernBertEmbeddings(
        (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
        (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (layers): ModuleList(
        (0): ModernBertEncoderLayer(
          (attn_norm): Identity()
          (attn): ModernBertAttention(
            (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
            (rotary_emb): ModernBertUnpaddedRotaryEmbedding(dim=64, base=160000.0, scale_base=None)
            (Wo): Linear(in_features=768, out_features=768, bias=False)
            (out_drop): Identity()
          )
          (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): ModernBertMLP(
            (Wi): Linear(in_features=768, out_features=2304, bias=False)
            (act): GELUActivation()
            (drop): D

In [12]:
import RoCoFT

RoCoFT.PEFT(model, method='column', rank=3) 
#targets=['key', 'value', 'dense', 'query'])

In [13]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # If predictions are logits or have extra dimensions, squeeze
    if predictions.ndim > 1:
        predictions = predictions.squeeze()

    mae = mean_absolute_error(labels, predictions)
    mse = mean_squared_error(labels, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(labels, predictions)
    
    # Define an "accuracy" for regression:
    # Example: within some threshold tolerance
    tolerance = 0.1  # you can change this
    acc = np.mean(np.abs(predictions - labels) < tolerance)

    pearson_corr, _ = pearsonr(predictions, labels)
    spearman_corr, _ = spearmanr(predictions, labels)

    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "Accuracy": acc,
        "R2": r2,
        "Pearson": pearson_corr,
        "Spearman's Rank": spearman_corr
    }


In [ ]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='dir',
    learning_rate=6e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.20,
    eval_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,

    data_collator=data_collator,
    compute_metrics=compute_metrics
)

[2025-05-05 02:04:29,128] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr

In [ ]:
trainer.train()

Got mask position:  tensor(-2, device='cuda:0')


Step,Training Loss,Validation Loss,Mae,Mse,Rmse,Accuracy,R2,Pearson,Spearman's rank
100,0.860100,0.320130,0.461172,0.320130,0.565800,0.121951,0.043091,0.227056,0.221171
200,0.331200,0.315117,0.452270,0.315117,0.561353,0.120711,0.058075,0.282844,0.282282
300,0.335700,0.320009,0.452211,0.320009,0.565693,0.119471,0.043453,0.306989,0.300344
400,0.322900,0.311424,0.446844,0.311424,0.558053,0.121951,0.069115,0.310244,0.303490
500,0.318400,0.297713,0.445014,0.297713,0.545630,0.126085,0.110099,0.335468,0.334384
600,0.298500,0.314216,0.462045,0.314216,0.560550,0.128979,0.060768,0.356391,0.350895
700,0.303100,0.294308,0.441544,0.294308,0.542501,0.128979,0.120277,0.356204,0.350045
800,0.288200,0.290713,0.437238,0.290713,0.539178,0.125672,0.131023,0.364127,0.356086
900,0.295200,0.291246,0.439101,0.291246,0.539672,0.121951,0.129429,0.360064,0.351700
1000,0.303800,0.296926,0.446907,0.296926,0.544909,0.134766,0.112451,0.370319,0.359735
